## TMM panel simulator adapted for cubical cavities and exterior necks

### Author: Alex Fanomezantsoa Rabearivony
### 2025

<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); 
            padding: 20px; border-radius: 10px; color: white; text-align: center;
            margin: 20px 0;">
    <h1>Acoustic Metamaterial Simulator</h1>
    <h3>Recycled Plastic Bottles - Helmholtz Resonators</h3>
    <g5>Interactive research tool for acoustic performance analysis</g5>
</div>

<div style="margin-left: 50px;">
<img src="IMAGES/3D cube.png" width="85%">
</div>


## Helmholtz Resonator Array – Interactive Simulation (TMM)

In this notebook, we investigate the **acoustic performance of a network of Helmholtz resonators** (plastic bottles) using the **Transfer Matrix Method (TMM)**.  
This approach allows us to simulate the **absorption coefficient** ($\alpha$) and the **transmission loss** (TL, in dB) of a periodic array of bottles.

---

### Theoretical Background

A **single Helmholtz resonator** has a resonance frequency given by:

$$
f_0 = \frac{c}{2 \pi} \sqrt{\frac{A}{V \, L_\text{eff}}}
$$

where:  
- $c$ = speed of sound in air (≈ 343 m/s)  
- $A = \pi r^2$ = neck cross-sectional area [m²]  
- $V$ = cavity volume [m³]  
- $L_\text{eff} = L + 1.7r$ = effective neck length [m]  

This expression provides the **theoretical resonance frequency** without losses.

---

### Absorption and Transmission Loss

For an incident plane wave, the **reflection coefficient** $R$ and the **transmission coefficient** $T$ can be obtained from the resonator impedance $Z$:

$$
R = \frac{Z - Z_0}{Z + Z_0}, \quad 
T = \frac{2Z_0}{Z + Z_0}
$$

with $Z_0 = \rho_0 c_0$ the characteristic impedance of air.  
Then:

- **Absorption coefficient**:  
$$
\alpha = 1 - |R|^2 - |T|^2
$$  

- **Transmission loss (TL)** in decibels:  
$$
TL = -20 \log_{10} |T|
$$  

---

### Analytical vs. Observed Resonance

 The **theoretical resonance frequency** $f_0$ does not always coincide with the **observed peak** of absorption or transmission loss.  
This is because **loss mechanisms** (viscous, thermal, radiation) and **array coupling effects** shift the resonance.

- The **red dashed line** on the plots indicates the **analytical $f_0$**.  
- The **measured peak** of $\alpha$ or TL may occur at a slightly different frequency, showing the impact of **damping and array interactions**.

---

## Limits of the **Transfer-Matrix Method (TMM) used here**

The TMM implementation in this notebook models each column as a cascade of propagation sections and a shunt admittance representing the resonators. This is a pragmatic and computationally efficient approach but carries specific limitations:

1. **One-dimensional (plane-wave) assumption in each propagation segment.** The TMM assumes plane waves propagate between scattering/shunt sites; evanescent higher transverse modes are neglected.  
2. **Heuristic coupling / homogenization.** The treatment of a column admittance as `rows * Y_hr * FF` is a pragmatic scaling (filling factor). It approximates coupling but does not fully account for near-field evanescent coupling or inter-resonator scattering.  
3. **Finite-size and edge effects.** TMM typically represents an infinite or periodically repeated geometry; finite panel edges can introduce diffraction and edge scattering that TMM does not capture accurately.  
4. **Single-frequency linear analysis.** The method computes linear steady-state responses; strongly nonlinear behavior (flow separation, vortex shedding at the neck) is outside scope.  
5. **Radiation and mounting conditions.** Radiation impedance and panel mounting (flexible backing, poroelastic supports) are simplified or absent; these can noticeably change absorption/TL in practice.  
6. **Numerical conditioning and singularities.** Near ideal (lossless) resonances the matrices can become ill-conditioned unless realistic damping (R0, viscous terms) or numerical regularization is included.  
7. **Parameter calibration required for quantitative predictions.** R0, Rscale and the filling factor FF are phenomenological in this model — for publication-grade predictions calibrate them by measurement or high-fidelity simulation.

---

### Key Observations

1. **Single bottle** → very low absorption (narrowband, small $\alpha$ values).  
2. **Array of bottles** → stronger collective effect, but resonance remains **narrowband**.  
3. **Losses** (via $R_0$ or viscous scaling) are necessary to reach significant absorption levels.  
4. **TL curve** is complementary to absorption: when $\alpha$ peaks, TL also increases.  

---
## Legend — variables controlled by widgets

<div align="center">
<img src="IMAGES/top cube.png" width="70%">
</div>
-
<div align="center">
<img src="IMAGES/Cross-section cube.png" width="50%">
</div>

- **cols** : number of columns in the array (integer).  
- **rows** : number of rows in the array (integer).  
- **neck r (mm)** : neck radius of each resonator, displayed in centimetres (converted to m for calculation).  
- **neck L (mm)** : neck (geometric) length, displayed in centimetres (converted to m for calculation).  
- **cav-side (mm)** : length of one side of the square cavity, defined in millimetres (converted to metres for computation).
- **fmax (Hz)** : maximum frequency shown in the plot (display zoom only).  
- **R0 (Pa·s/m)** : frequency-independent lumped resistance used to represent losses (user tuneable).  
- **R scale** : multiplicative scaling of the viscous resistance model.  
- **M_mem (mg)** : optional added mass (membrane mass) in milligrams.

This interactive setup allows exploring **how geometry and material properties influence absorption and TL**.  

---

In [1]:
# TMM panel simulator adapted for cubical cavities and exterior necks
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown

# --- physical constants ---
rho0 = 1.21
c0 = 343.0
Z0 = rho0 * c0
mu = 1.84e-5

# --- helper functions ---
def helmholtz_f0(A, V, L, r, c=c0):
    # A: neck area (m^2), V: cavity volume (m^3), L: physical neck length (m), r: neck radius (m)
    L_eff = L + 1.7*r
    return (c/(2*np.pi)) * np.sqrt( A / (V * L_eff) )

def viscous_delta(f):
    omega = 2*np.pi*np.maximum(f,1e-8)
    return np.sqrt(2*mu/(rho0*omega))

def R_viscous(f, r):
    # r: radius (m) (use r_eq or hydraulic radius as appropriate)
    delta = viscous_delta(f)
    r = np.maximum(r, 1e-6)
    return Z0 * 2.0 * (delta / r)

def Z_helmholtz(f, A, V, L, r, R0=0.0, Rscale=1.0, mem_mass_kg=0.0):
    """Impedance of one HR (vectorized).
       A: neck area (m^2)
       V: cavity volume (m^3)
       L: physical neck length (m)  -- Z_helmholtz computes L_eff = L + 1.7*r internally
       r: neck radius (m)
    """
    omega = 2*np.pi*np.maximum(f,1e-8)
    L_eff = L + 1.7*r
    # mass of air in neck (inertance)
    M_air = rho0 * L_eff / (A + 1e-12)
    # optional extra mass (membrane) spread over area -> convert to comparable inertance-like term
    M_extra = 0.0
    if mem_mass_kg > 0:
        # convert mass (kg) spread over the neck area to an effective inertial term (heuristic)
        M_extra = mem_mass_kg / (A + 1e-12)
    M = M_air + M_extra
    C = V / (rho0 * c0**2)
    Rv = Rscale * R_viscous(f, r)
    Z = R0 + Rv + 1j*omega*M - 1j/(omega*C)
    return Z

def compute_array_response(freq, rows, cols, d, A_neck, V_cav, L_neck, r_neck,
                           FF_geom, R0, Rscale, mem_mass_kg):
    """Compute alpha, TL, and individual HR impedance for an array of HRs.
       d: pitch/spacing between centers (m)
       FF_geom: filling factor (geometric) used only for reporting (we still use rows*Y_hr)
    """
    omega = 2*np.pi*freq
    k0 = omega / c0
    Z_hr = Z_helmholtz(freq, A_neck, V_cav, L_neck, r_neck, R0=R0, Rscale=Rscale, mem_mass_kg=mem_mass_kg)
    Y_hr = 1.0 / (Z_hr + 1e-18)
    # collective admittance per column (rows stacked along transverse direction)
    Y_col = rows * Y_hr  # FF handled geometrically externally if needed
    kd = k0 * (d/2.0)
    cosk = np.cos(kd); sink = np.sin(kd)
    P11 = cosk; P12 = 1j * Z0 * sink
    P21 = 1j * (1.0/Z0) * sink; P22 = cosk
    alpha = np.zeros_like(freq, dtype=float)
    TL = np.zeros_like(freq, dtype=float)
    for i in range(len(freq)):
        P = np.array([[P11[i], P12[i]],[P21[i], P22[i]]], dtype=complex)
        S = np.array([[1.0, 0.0],[Y_col[i], 1.0]], dtype=complex)
        Mcell = P @ S @ P
        # global matrix for 'cols' repeated cells (assumes periodic repeat along columns)
        Mglob = np.linalg.matrix_power(Mcell, cols)
        A = Mglob[0,0]; B = Mglob[0,1]; Cmat = Mglob[1,0]; D = Mglob[1,1]
        T = 2.0 / (A + B/Z0 + Cmat*Z0 + D)
        R = (A + B/Z0 - Cmat*Z0 - D) / (A + B/Z0 + Cmat*Z0 + D)
                # safer TL calculation: clip transmission magnitude to avoid -inf/huge dB from numerical zeros
        Tmag = np.clip(np.abs(T), 1e-12, 1.0)   # floor at 1e-12 (≈ -240 dB) to avoid numerical explosion
        TL[i] = -20.0 * np.log10(Tmag)

        alpha[i] = np.clip(1.0 - np.abs(R)**2 - np.abs(T)**2, 0.0, 1.0)
    return alpha, TL, Z_hr

# ----------------- widgets -----------------
cols_w = widgets.IntSlider(value=15, min=1, max=24, step=1, description='cols', continuous_update=False)
rows_w = widgets.IntSlider(value=15, min=1, max=24, step=1, description='rows', continuous_update=False)

# neck radius slider kept as in your original code (cm input)
neck_r_w = widgets.FloatSlider(value=10.0, min=2.0, max=50.0, step=0.5, description='neck r (mm)', continuous_update=False)
# physical neck length in mm (tube length, outside the front face)
neck_L_w = widgets.FloatSlider(value=10.0, min=2.0, max=50.0, step=1.0, description='neck L (mm)', continuous_update=False)

# cavity side (interior cube side) in mm (replaced bottle diameter)
cavity_side_w = widgets.FloatSlider(value=66.0, min=20.0, max=150.0, step=1.0, description='cav-side(mm)', continuous_update=False)

R0_w = widgets.FloatSlider(value=415.0, min=0.0, max=500.0, step=5.0, description='R0', continuous_update=False)
Rscale_w = widgets.FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description='R scale', continuous_update=False)
mass_w = widgets.FloatSlider(value=0.0, min=0.0, max=500.0, step=1.0, description='M_mem (mg)', continuous_update=False)

fmax_w = widgets.IntSlider(value=1000, min=200, max=8000, step=100, description='fmax (Hz)', continuous_update=False)

run_btn = widgets.Button(description='Run simulation', button_style='success')
reset_btn = widgets.Button(description='Reset', button_style='warning')

out = widgets.Output(layout={'border':'1px solid black'})

# internal fixed calc grid
NPOINTS = 3000

def on_run(b):
    with out:
        clear_output(wait=True)
        # read widgets & convert units
        cols = cols_w.value
        rows = rows_w.value
        # neck radius given in cm -> convert to meters
        r_neck = neck_r_w.value / 1000.0  # mm → m
        # neck length given in mm -> convert to meters
        L_neck = neck_L_w.value / 1000.0
        # cavity side given in mm -> convert to meters (interior dimension)
        cavity_side_m = cavity_side_w.value / 1000.0
        # cavity volume (cube interior)
        V_l = cavity_side_m**3   # m^3
        V = V_l
        # geometric area of neck (m^2)
        A_neck = np.pi * r_neck**2
        # total neck area for all HRs
        total_neck_area = rows * cols * A_neck
        # panel pitch: assume cells are tiled without gap, pitch = cavity_side_m
        pitch = cavity_side_m
        panel_area = (cols * pitch) * (rows * pitch)
        FF_geom = np.clip(total_neck_area / panel_area, 1e-8, 1.0)
        # parameters R and Rscale
        R0 = R0_w.value
        Rscale = Rscale_w.value
        mem_mass_kg = mass_w.value * 1e-6
        # spacing/period used in TMM (use cavity side as 'd')
        d = cavity_side_m
        # estimate single HR resonance with L_eff = L + 1.7*r
        f_res_est = helmholtz_f0(A_neck, V, L_neck, r_neck)
        # frequency grid for calculation (NPOINTS)
        # choose upper bound reasonably: at least 4*f_res_est but at least 8000 (kept from original logic)
        fcalc_upper = max(int(np.ceil(4*f_res_est)), 8000)
        freq = np.linspace(0.1, fcalc_upper, NPOINTS)
        # compute array response
        alpha, TL, Zhr = compute_array_response(freq, rows, cols, d, A_neck, V, L_neck, r_neck,
                                               FF_geom, R0, Rscale, mem_mass_kg)
        # display only up to fmax slider (visual zoom only)
        fmax = fmax_w.value
        idx = freq <= fmax
        f_show = freq[idx]; alpha_show = alpha[idx]; TL_show = TL[idx]
        alpha_max = np.max(alpha); f_alpha = freq[np.argmax(alpha)]
        TL_max = np.max(TL); f_TL = freq[np.argmax(TL)]
        # plotting
        fig, ax = plt.subplots(1,2, figsize=(12,4))
        ax[0].plot(f_show, alpha_show, lw=1.4); ax[0].axvline(f_res_est, color='b', ls='--', label=f"single f0={f_res_est:.1f} Hz")
        ax[0].set_xlabel('Freq (Hz)'); ax[0].set_ylabel('Absorption α'); ax[0].set_ylim(-0.02,1.05); ax[0].grid(True)
        ax[0].legend(); ax[0].set_title(f"α_max={alpha_max:.2f} at {f_alpha:.0f} Hz")
        ax[1].plot(f_show, TL_show, lw=1.4); ax[1].axvline(f_res_est, color='b', ls='--')
        ax[1].set_xlabel('Freq (Hz)'); ax[1].set_ylabel('Transmission Loss (dB)'); ax[1].grid(True)
        TL_max_display = np.minimum(TL_max, 120.0)
        ax[1].set_title(f"TL_max={TL_max_display:.1f} dB at {f_TL:.0f} Hz")

        plt.suptitle(f"Array: cols={cols}, rows={rows}, FF(geom)={FF_geom:.3f}")
        plt.show()

        # --- Panel geometric dimensions (computed from cols, rows, pitch) ---
        panel_length_cm = cols * pitch * 100.0    # length along columns (cm)
        panel_width_cm  = rows * pitch * 100.0    # width along rows (cm)
        panel_thickness_cm = (cavity_side_m + 0.002 + 0.002) * 100.0  # cavity interior + front + back (cm)

        print(f"Panel dimensions (cm): length = {panel_length_cm:.1f} cm, width = {panel_width_cm:.1f} cm, thickness ≈ {panel_thickness_cm:.1f} cm")
        print(f"Panel area (m^2) = {panel_area:.4f}, total neck area (m^2) = {total_neck_area:.6f}")
        print(f"Single HR approx f0 = {f_res_est:.1f} Hz (L_phys={L_neck*1000:.1f} mm, r={r_neck*1000:.2f} mm)")
        # compute L_eff for display
        r_eq = np.sqrt(A_neck/np.pi)
        L_eff = L_neck + 1.7 * r_eq
        print(f"Neck area A_neck = {A_neck*1e6:.2f} mm², neck radius (mm) = {r_neck*1000:.2f}, L_eff = {L_eff*1000:.2f} mm")
        print(f"Volume cavity (L) = {V*1000:.4f} L")
        print(f"Loss model: R0={R0:.1f} Pa·s/m, Rscale={Rscale:.2f}, M_mem={mass_w.value:.1f} mg")
        print("Note: FF is computed geometrically (FF_geom). If you need an empirical tuning, adjust R0/Rscale or the neck geometry.")

                # ------------------ Save summary row to CSV ------------------
        import os
        import pandas as pd

        # metrics (raw)
        alpha_max_val = float(alpha_max)
        TL_max_val = float(TL_max)           # valeur brute (peut être très élevée si T ~ 0)
        fo_val = float(f_res_est)            # fréquence d'estimation simple
        # pour sécurité lisible on crée aussi une valeur TL "capped" utile pour lecture humaine
        TL_max_capped = float(np.minimum(TL_max_val, 120.0))  # on plafonne à 120 dB pour lisibilité

        # build summary dict (columns demanded)
        summary = {
            "cav-side(mm)": float(cavity_side_w.value),
            "neck_r(mm)": float(neck_r_w.value),
            "neck_L(mm)": float(neck_L_w.value),
            "R0": float(R0),
            "R_scale": float(Rscale),
            "M_mem(mg)": float(mass_w.value),
            "alpha_max": alpha_max_val,
            "TL_max_raw_dB": TL_max_val,
            "TL_max_capped_dB": TL_max_capped,
            "f0_est_Hz": fo_val
        }

        csv_file = "Cubical_results.csv"
        # if file doesn't exist, write header; else append
        if not os.path.exists(csv_file):
            df0 = pd.DataFrame([summary])
            df0.to_csv(csv_file, index=False)
        else:
            df0 = pd.DataFrame([summary])
            df0.to_csv(csv_file, mode='a', header=False, index=False)

        print(f"✅ Summary appended to '{csv_file}' (one row per run). TL_max (raw) = {TL_max_val:.1f} dB, capped shown = {TL_max_capped:.1f} dB")
  

# --- Reset handler ---
def on_reset(b):
    cols_w.value = 15
    rows_w.value = 15
    neck_r_w.value = 10.0
    neck_L_w.value = 10.0
    cavity_side_w.value = 66.0
    R0_w.value = 415.0
    Rscale_w.value = 1.0
    mass_w.value = 0.0
    fmax_w.value = 1000
    with out:
        clear_output(wait=True)
        print("🔄 Parameters reset to default values.")

# --- Interface layout ---
controls = widgets.VBox([
    widgets.HBox([cols_w, rows_w, cavity_side_w]),
    widgets.HBox([neck_r_w, neck_L_w, fmax_w]),
    widgets.HBox([R0_w, Rscale_w, mass_w]),
    widgets.HBox([ run_btn, reset_btn])
])

display(controls, out)

# --- Bind buttons ---
run_btn.on_click(on_run)
reset_btn.on_click(on_reset)


Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…